In [1]:
%pwd

'c:\\Users\\satish.hiremath\\OneDrive - o9 Solutions\\Desktop\\python\\e2e_mlops_dimond_price_prediction\\e2e_mlops_dimond_price_prediction\\experiments'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\satish.hiremath\\OneDrive - o9 Solutions\\Desktop\\python\\e2e_mlops_dimond_price_prediction\\e2e_mlops_dimond_price_prediction'

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file_train: Path
    local_data_file_test:Path
    unzip_dir: Path

In [9]:
from src.DiamondPricePredictor.constants import *
from src.DiamondPricePredictor.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        print(params_filepath)
        print(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file_train=config.local_data_file_train,
            local_data_file_test=config.local_data_file_test,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [11]:
import os
import urllib.request as request
import zipfile
from DiamondPricePredictor.logger import logging
from DiamondPricePredictor.utils.common import get_size

import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        

    
    def download_train_file(self):
        if not os.path.exists(self.config.local_data_file_train):
            api=KaggleApi()
            api.authenticate()
            api.competition_download_file('playground-series-s3e8',file_name='train.csv',path=self.config.root_dir)
            logging.info("Train file download completed:")
        else:
            logging.info(f"File already exists of size: {get_size(Path(self.config.local_data_file_train))}")  

    def download_test_file(self):
        if not os.path.exists(self.config.local_data_file_test):
            api=KaggleApi()
            api.authenticate()
            api.competition_download_file('playground-series-s3e8',file_name='test.csv',path=self.config.root_dir)
            logging.info("Test File Download completed")
        else:
            logging.info(f"File already exists of size: {get_size(Path(self.config.local_data_file_test))}")  
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file_train, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logging.info("Train file extraction completed")
        with zipfile.ZipFile(self.config.local_data_file_test, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logging.info("Test file extraction completed")

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_train_file()
    data_ingestion.download_test_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[ 2024-02-02 13:09:43,181] 31 root-INFO -yaml file: config\config.yaml loaded successfully
params.yaml
config\config.yaml
[ 2024-02-02 13:09:43,190] 31 root-INFO -yaml file: params.yaml loaded successfully
[ 2024-02-02 13:09:43,193] 51 root-INFO -created directory at: artifacts
[ 2024-02-02 13:09:43,195] 51 root-INFO -created directory at: artifacts/data_ingestion


NameError: name 'api' is not defined